<a href="https://colab.research.google.com/github/fatemafaria142/Evaluating-Generated-Recipes-Against-Reference-Recipes/blob/main/Recipe_Generation_using_GPT_2_Medium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Read dataset from google drive**
### **Dataset Link: https://huggingface.co/datasets/recipe_nlg**
### **Paper Link: https://aclanthology.org/2020.inlg-1.4/**

In [2]:
import pandas as pd
dataset = pd.read_csv('/content/drive/MyDrive/Recipe Generation/full_dataset.csv')
dataset.head()

,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


# **Length of the main dataset**

In [3]:
len(dataset)

2231142

# **Take subset of the main dataset**

In [4]:
# Extracting the first 15,000 examples for train and test
data = dataset.head(15000)

# Saving the subset as a CSV file
data.to_csv('/content/drive/MyDrive/Recipe Generation/recipes.csv', index=False)


# **Taking 90% for train dataset and 10% for test dataset**

In [5]:
import re
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the dataset from the CSV file
data = pd.read_csv('/content/drive/MyDrive/Recipe Generation/recipes.csv')

def build_text_files(data_csv, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for index, row in data_csv.iterrows():
        summary = str(row['directions']).strip()
        summary = re.sub(r"[\[\]\"]", "", summary)  # Remove square brackets and quotation marks
        summary = re.sub(r"\s+", " ", summary)  # Replace multiple spaces with a single space
        data += summary + "  "
    f.write(data)

train, test = train_test_split(data, test_size=0.10)

build_text_files(train, '/content/drive/MyDrive/Recipe Generation/train_dataset.txt')
build_text_files(test, '/content/drive/MyDrive/Recipe Generation/test_dataset.txt')

print("Train dataset length:", len(train))
print("Test dataset length:", len(test))


Train dataset length: 13500
Test dataset length: 1500


# **Text Preprocessing**

In [6]:
def read_text_file(file_path, num_examples=5):
    with open(file_path, 'r') as file:
        content = file.read()
        examples = content.split('  ')  # Assuming '  ' is the separator between examples
        for i, example in enumerate(examples[:num_examples]):
            print(f"Example {i+1}: {example}")

train_file_path = '/content/drive/MyDrive/Recipe Generation/train_dataset.txt'
test_file_path = '/content/drive/MyDrive/Recipe Generation/test_dataset.txt'

print("Examples from the Train Dataset:")
read_text_file(train_file_path)

print("\nExamples from the Test Dataset:")
read_text_file(test_file_path)


Examples from the Train Dataset:
Example 1: Beat sugar, margarine, vanilla and egg in a medium bowl until light and fluffy., Stir in remaining ingredients except caramel icing., Cover and refrigerate 1 hour, until chilled., Heat oven to 375\u00b0., Roll dough 1/8-inch thick on lightly floured surface., Cut into cookies., Bake on greased cookie sheet until brown.
Example 2: Line serving bowl with lettuce leaves., Arrange layers of avocado and orange slices on top., Combine mayonnaise, orange juice, paprika and salt in small bowl and mix well., Pour over salad.
Example 3: Line bottom of baking dish with 6 slices of Armour dried beef. Top with boneless chicken breasts.
Example 4: In large bowl, mix butter, sugar, egg yolk, vanilla, flour and 1/2 cup nuts., Blend well., Spread in ungreased jelly roll pan 15 x 10-inch., Beat egg whites until frothy., Spread over mix. Sprinkle with colored sugar and 1/2 cup nuts., Bake at 350\u00b0 for 25 to 30 minutes until golden brown., Cool and cut into 

In [7]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 7.2 MB/s eta 0:00:00


In [8]:
!pip install accelerate -U

In [9]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 42.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


# **GPT-2 Medium and its tokenizer**
### **https://huggingface.co/gpt2-medium**

In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained('gpt2-medium')
model = AutoModelForCausalLM.from_pretrained('gpt2-medium')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [11]:
train_path = '/content/drive/MyDrive/Recipe Generation/train_dataset.txt'
test_path = '/content/drive/MyDrive/Recipe Generation/test_dataset.txt'

# **Prepare the dataset and build a TextDataset**
* The next step is to extract the instructions from all recipes and build a TextDataset. The TextDataset is a custom implementation of the Pytroch Dataset class implemented by the transformers library.

In [12]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


# **1st time train - Initialize Trainer with TrainingArguments**

In [14]:
'''
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./gpt2-englishchef", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=8, # batch size for training
    per_device_eval_batch_size=8,  # batch size for evaluation
    eval_steps = 500, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)
'''

In [15]:
'''
trainer.train()
'''

Step,Training Loss
500,2.598800
1000,2.267700
1500,2.152800
2000,2.027800


TrainOutput(global_step=2367, training_loss=2.2221852710693257, metrics={'train_runtime': 2159.9445, 'train_samples_per_second': 8.763, 'train_steps_per_second': 1.096, 'total_flos': 4394379511332864.0, 'train_loss': 2.2221852710693257, 'epoch': 3.0})

In [16]:
'''
trainer.save_model()
'''

# **Loading the saved model and train it again**

* The Trainer class provides an API for feature-complete training. It is used in most of the example scripts from Huggingface. Before we can instantiate our Trainer we need to download our GPT-2 model and create a TrainingArguments to access all the points of customization during training. In the TrainingArguments, we can define the Hyperparameters we are going to use in the training process like our learning_rate, num_train_epochs, or per_device_train_batch_size.


In [17]:
from transformers import Trainer, TrainingArguments
# Load the previously trained model
model_path = "./gpt2-englishchef"  # Replace this with the path to your saved model
model = AutoModelForCausalLM.from_pretrained(model_path)

# Define your training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-englishchef",  # The output directory
    overwrite_output_dir=True,  # Overwrite the content of the output directory
    num_train_epochs=5,  # Number of training epochs
    per_device_train_batch_size=8,  # Batch size for training
    per_device_eval_batch_size=8,  # Batch size for evaluation
    eval_steps=500,  # Number of update steps between two evaluations
    save_steps=800,  # Save model checkpoint after # steps
    warmup_steps=500,  # Number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
)

# Initialize Trainer with your loaded model and training arguments
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)


# **Train the model**

In [18]:
# Start training to continue from where it left off
trainer.train()

Step,Training Loss
500,1.957500
1000,1.826900
1500,1.828800
2000,1.754400
2500,1.792200
3000,1.732100
3500,1.691200


Checkpoint destination directory ./gpt2-englishchef/checkpoint-800 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./gpt2-englishchef/checkpoint-1600 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=3945, training_loss=1.7824314518724256, metrics={'train_runtime': 3670.3909, 'train_samples_per_second': 8.594, 'train_steps_per_second': 1.075, 'total_flos': 7323965852221440.0, 'train_loss': 1.7824314518724256, 'epoch': 5.0})

# **Save the model**

In [19]:
# Save the model after training
trainer.save_model("./gpt2-englishchef")

# **Test the model**
* To test the model we are going to use another highlight of the transformers library called pipeline. Pipelines are objects that offer a simple API dedicated to several tasks, among others also text-generation

In [20]:
from transformers import pipeline

In [21]:
chef = pipeline('text-generation',model='./gpt2-englishchef', tokenizer='gpt2')

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [22]:
chef('Beat together eggs, sugar and flour.')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Beat together eggs, sugar and flour. Add buttermilk, milk, vanilla and baking powder. Combine well. Spread on ungreased baking sheet. Sprinkle with sugar and then cinnamon. Bake at 375\\u00b0 for about 15'}]

In [23]:
chef(' Stir Velveeta and salsa in saucepan over low heat until Velveeta is melted.')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': ' Stir Velveeta and salsa in saucepan over low heat until Velveeta is melted. Place meat mixture in 11 x 8 x 2-inch baking dish; place foil on top., Combine Cheddar cheese, tomato sauce, oregano and'}]

In [24]:
chef('In one bowl, combine all ingredients except frosting.')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In one bowl, combine all ingredients except frosting. Cut each biscuit in half and lay flat., Place a layer of biscuit dough on an ungreased cookie sheet., Combine cream cheese, powdered sugar and Cool Whip., Add the biscuit'}]

In [25]:
chef(' Mix all ingredients except soda.')[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


' Mix all ingredients except soda.  Place drained cucumbers in a 1/2-quart glass baking dish., Pour soup and broth over cucumbers., Toss with dressing., Combine remaining ingredients and mix well., Pour over cucumbers., Refrigerate'

# **Evaluate perplexity on the test dataset**

In [26]:
# Evaluate perplexity on the test dataset
eval_result = trainer.evaluate(eval_dataset=test_dataset)
print("Perplexity:", eval_result['eval_loss'])

Perplexity: 2.1657066345214844


# **Generating Recipes without any prompt**

In [27]:
from transformers import pipeline

# Initialize the text generation pipeline
chef = pipeline('text-generation', model='./gpt2-englishchef', tokenizer='gpt2')

# Function to generate recipes and save them
def generate_recipes(chef_pipeline, num_recipes: int):
    generated_recipes = []
    for _ in range(num_recipes):
        generated_recipe = chef_pipeline("Recipe: ")  # You can modify the initial prompt if needed
        generated_recipes.append(generated_recipe[0]['generated_text'])
    return generated_recipes

# Generate recipes
num_recipes_to_generate = 5  # Define the number of recipes to generate
generated_recipes = generate_recipes(chef, num_recipes_to_generate)

# Display the generated recipes
for i, recipe in enumerate(generated_recipes, start=1):
    print(f"Generated Recipe {i}:\n{recipe}\n")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Recipe 1:
Recipe:  You will need an 8 x 8 x 2-inch pan., Cook rice according to package directions., Mix margarine, cheese and sugar together and pour on top of rice and other ingredients., Bake at 350\u00b0 for

Generated Recipe 2:
Recipe:  Grease and flour 9 x 13-inch pan., Grease bottoms only of all. Using mixer or hand beater prepare egg whites according to package directions., Add to dry ingredients and beat well., Stir in cracker crumbs

Generated Recipe 3:
Recipe:  Mix margarine, sugar and eggs together., Add flour, sifted dry ingredients and vanilla to sugar mixture., Add bananas and coconut, just enough so it will keep sticking together when it comes out of the bowl., Add sliced peaches

Generated Recipe 4:
Recipe:  Combine first 6 ingredients., Stir in 1-inch jar of ginger ale and shake., Makes 2 gallons.  Combine first 4 ingredients and pat into a 9 by 13 inch pan., Bake at 350\u00b0 for 15 minutes

Generated Recipe 5:
Recipe:  Cream butter and sugars., Add salt, v

# **Examples from Test Dataset**

In [28]:
def read_text_file(file_path, num_examples=5):
    with open(file_path, 'r') as file:
        content = file.read()
        examples = content.split('  ')  # Assuming '  ' is the separator between examples
        for i, example in enumerate(examples[:num_examples]):
            print(f"Example {i+1}: {example}")


test_file_path = '/content/drive/MyDrive/Recipe Generation/test_dataset.txt'

print("\nExamples from the Test Dataset:")
read_text_file(test_file_path)


Examples from the Test Dataset:
Example 1: Stagger all of the ingredients into a large, covered, casserole., Cover and bake in a slow oven at 225\u00b0 to 250\u00b0 for about 6 to 7 hours (test meat and vegetables for doneness)., Stir once after cooking 3 to 4 hours., Makes 6 servings.
Example 2: Mix and freeze all ingredients., Take out about 3 hours before serving and pour in ginger ale.
Example 3: Combine mustard, sugar, salt and a generous amount of pepper. Mix in garlic., Add vinegar and lemon juice and stir thoroughly. Add oil and mix well., Stir or shake vigorously just before using. Makes about 3 cups or 24 servings of 2 tablespoons each.
Example 4: Cut shortening into dry mixture., Add enough milk to make a fairly wet dough., Place on floured board., Cut into rounds about 1/2 inch high and place close together in pan (greased)., Bake in hot oven until brown.
Example 5: Wash beans and soak overnight in water to cover., The next morning, if necessary, add more water to cover., 

# **Generating Recipes with prompt from test dataset**

In [29]:
from transformers import pipeline
import random

test_dataset_file = '/content/drive/MyDrive/Recipe Generation/test_dataset.txt'

# Initialize the text generation pipeline
chef = pipeline('text-generation', model='./gpt2-englishchef', tokenizer='gpt2')

# Read your test dataset
with open(test_dataset_file, 'r') as file:
    test_content = file.read()

# Split the content into individual recipes based on the delimiter ('  ')
test_recipes = test_content.split('  ')

# Function to generate recipes based on the test dataset
def generate_recipes_from_test_dataset(chef_pipeline, test_dataset, num_recipes: int):
    generated_recipes = []
    for i in range(num_recipes):
        # Randomly select a recipe from the test dataset
        random_recipe = random.choice(test_dataset)
        # Extract the first 6/7 words from the recipe as the prompt
        words = random_recipe.split()[:6]  # Change the number here to select the desired number of words
        prompt = ' '.join(words)
        generated_recipe = chef_pipeline(prompt)
        generated_recipes.append(generated_recipe[0]['generated_text'])
    return generated_recipes

# Generate recipes based on the test dataset
num_recipes_to_generate = 5  # Define the number of recipes to generate
generated_recipes_from_test = generate_recipes_from_test_dataset(chef, test_recipes, num_recipes_to_generate)

# Display the generated recipes
for i, recipe in enumerate(generated_recipes_from_test, start=1):
    print(f"Generated Recipe {i}:\n{recipe}\n")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Recipe 1:
Preheat oven to 350 degrees., Bake on 15 hour preheated oven.  Mix all ingredients and drop by teaspoonfuls into hot oil and fry until dark brown.  Bring water to a boil., Add ice cream to water and continue boiling

Generated Recipe 2:
Sprinkle nuts in bottom of well-greased 10-inch Bundt pan., Pour 2 cups mixed nuts over top., Sprinkle remaining 2 cups mixed nuts on top., Spread cream cheese evenly over top., Bake at 350\u00b0

Generated Recipe 3:
Add beans and brown sugar to taste., Add remaining ingredients and simmer covered for 1 hour.  Combine first 5 ingredients in large bowl., Mix well., Place 1 tablespoon mixture in each well-beaten egg., Bake at 325\u00b0

Generated Recipe 4:
Combine ingredients in order and bake at 400\u00b0 for 30 minutes., This is a great dessert for a party., It is also great for a potluck.  Mix all ingredients and shape into balls., Fry in a hot oil

Generated Recipe 5:
Combine all ingredients, except ginger ale., Pour into Jell-O mo

# **List of 1st 20 examples from Test Dataset**

In [30]:
def read_text_file(file_path, num_examples=5):
    with open(file_path, 'r') as file:
        content = file.read()
        examples = content.split('  ')  # Assuming '  ' is the separator between examples
        return examples[:num_examples]

test_file_path = '/content/drive/MyDrive/Recipe Generation/test_dataset.txt'

print("\nExamples from the Test Dataset:")
first_20_examples = read_text_file(test_file_path, num_examples=20)

# Display the first 20 examples
for i, example in enumerate(first_20_examples, start=1):
    print(f"Example {i}: {example}")

# Store the first 20 examples in a list
list_of_20_examples = first_20_examples


Examples from the Test Dataset:
Example 1: Stagger all of the ingredients into a large, covered, casserole., Cover and bake in a slow oven at 225\u00b0 to 250\u00b0 for about 6 to 7 hours (test meat and vegetables for doneness)., Stir once after cooking 3 to 4 hours., Makes 6 servings.
Example 2: Mix and freeze all ingredients., Take out about 3 hours before serving and pour in ginger ale.
Example 3: Combine mustard, sugar, salt and a generous amount of pepper. Mix in garlic., Add vinegar and lemon juice and stir thoroughly. Add oil and mix well., Stir or shake vigorously just before using. Makes about 3 cups or 24 servings of 2 tablespoons each.
Example 4: Cut shortening into dry mixture., Add enough milk to make a fairly wet dough., Place on floured board., Cut into rounds about 1/2 inch high and place close together in pan (greased)., Bake in hot oven until brown.
Example 5: Wash beans and soak overnight in water to cover., The next morning, if necessary, add more water to cover., 

In [31]:
print(list_of_20_examples)

['Stagger all of the ingredients into a large, covered, casserole., Cover and bake in a slow oven at 225\\u00b0 to 250\\u00b0 for about 6 to 7 hours (test meat and vegetables for doneness)., Stir once after cooking 3 to 4 hours., Makes 6 servings.', 'Mix and freeze all ingredients., Take out about 3 hours before serving and pour in ginger ale.', 'Combine mustard, sugar, salt and a generous amount of pepper. Mix in garlic., Add vinegar and lemon juice and stir thoroughly. Add oil and mix well., Stir or shake vigorously just before using. Makes about 3 cups or 24 servings of 2 tablespoons each.', 'Cut shortening into dry mixture., Add enough milk to make a fairly wet dough., Place on floured board., Cut into rounds about 1/2 inch high and place close together in pan (greased)., Bake in hot oven until brown.', 'Wash beans and soak overnight in water to cover., The next morning, if necessary, add more water to cover., Add salt and simmer for 1 1/2 to 2 hours or until the beans are very ten

In [32]:
print(len(list_of_20_examples))

20


In [33]:
from transformers import pipeline

test_dataset_file = '/content/drive/MyDrive/Recipe Generation/test_dataset.txt'

# Initialize the text generation pipeline
chef = pipeline('text-generation', model='./gpt2-englishchef', tokenizer='gpt2')

# Read your test dataset
with open(test_dataset_file, 'r') as file:
    test_content = file.read()

# Split the content into individual recipes based on the delimiter ('  ')
test_recipes = test_content.split('  ')

# Function to generate recipes based on the test dataset
def generate_recipes_from_test_dataset(chef_pipeline, test_dataset, num_recipes: int):
    generated_recipes = []
    for i in range(num_recipes):
        # Extract the first 6 words from the recipe as the prompt
        words = test_dataset[i].split()[:6] if i < len(test_dataset) else test_dataset[-1].split()[:6]
        prompt = ' '.join(words)
        generated_recipe = chef_pipeline(prompt, max_length=128)  # Adjust max_length as needed
        generated_recipes.append(generated_recipe[0]['generated_text'])
    return generated_recipes

# Generate recipes based on the test dataset
num_recipes_to_generate = 20  # Define the number of recipes to generate
generated_recipes_from_test = generate_recipes_from_test_dataset(chef, test_recipes, num_recipes_to_generate)

# Display the generated recipes
for i, recipe in enumerate(generated_recipes_from_test, start=1):
    print(f"Generated Recipe {i}:\n{recipe}\n")

# Save the generated recipes in a list
list_of_generated_recipes = generated_recipes_from_test


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Generated Recipe 1:
Stagger all of the ingredients into a medium-large bowl or a glass measuring cup., Let stand until almost firm; do not overcook., Using a long knife, cut shortening into crumbs into a bowl., In a large bowl, combine sugar, molasses, flour, cinnamon and salt., Using a pastry blender or two knives, cut butter into crumbs until mixture resembles coarse meal (mixture will be sticky)., With your hands, form mixture by gently pressing a bit of mixture into the fingers and thumb of an electric mixer or by hand until all is moist., Turn onto a floured surface., Knead until

Generated Recipe 2:
Mix and freeze all ingredients., Take out before serving.  Heat oven to 350\u00b0. Cook ground beef and onion in skillet until beef is brown; drain well. Stir in Cheddar cheese., Spread 1 1/2 cups American cheese into 2nd square of 9 x 13-inch baking dish., Place chicken over cheese., Pour chicken mixture over chicken., Add remaining cheese on top., Add sliced mushrooms., Bake 30 minu

In [34]:
print(list_of_generated_recipes)

['Stagger all of the ingredients into a medium-large bowl or a glass measuring cup., Let stand until almost firm; do not overcook., Using a long knife, cut shortening into crumbs into a bowl., In a large bowl, combine sugar, molasses, flour, cinnamon and salt., Using a pastry blender or two knives, cut butter into crumbs until mixture resembles coarse meal (mixture will be sticky)., With your hands, form mixture by gently pressing a bit of mixture into the fingers and thumb of an electric mixer or by hand until all is moist., Turn onto a floured surface., Knead until', 'Mix and freeze all ingredients., Take out before serving.  Heat oven to 350\\u00b0. Cook ground beef and onion in skillet until beef is brown; drain well. Stir in Cheddar cheese., Spread 1 1/2 cups American cheese into 2nd square of 9 x 13-inch baking dish., Place chicken over cheese., Pour chicken mixture over chicken., Add remaining cheese on top., Add sliced mushrooms., Bake 30 minutes or until bubbly., Great for rib

In [35]:
print(len(list_of_generated_recipes))

20


# **Evaluating Generated Recipes Against Reference Recipes**

In [36]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=96fafd3fdf03129b45bb429c3c7255cb15b6ea432e5098a18f6389859c1f6d05
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [37]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.0 MB/s eta 0:00:00


In [38]:
!pip install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 34.8 MB/s eta 0:00:00


In [39]:
from evaluate import load
# Load the evaluation metric for Character Error Rate (CER) and Word Error Rate (WER) and Exact Match(em)
cer_metric = load("cer")
wer_metric = load("wer")
meteor = load('meteor')
exact_match_metric = load("exact_match")

# Load BLEU and ROUGE metrics
bleu_metric = load("bleu")
rouge_metric = load('rouge')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [40]:
# Calculate Character Error Rate (CER) and Word Error Rate (WER)
results_CER = cer_metric.compute(predictions=list_of_generated_recipes, references=list_of_20_examples)
results_WER = wer_metric.compute(predictions=list_of_generated_recipes, references=list_of_20_examples)

# Calculate Exact Match (EM) and METEOR(M)
results_em = exact_match_metric.compute(predictions=list_of_generated_recipes, references=list_of_20_examples)
results_met = meteor.compute(predictions=list_of_generated_recipes, references=list_of_20_examples)

# Calculate Bilingual Evaluation Understudy (BLEU)
results_bleu = bleu_metric.compute(predictions=list_of_generated_recipes, references=list_of_20_examples)

# **Printing evaluation metrics**

In [41]:
print("Character Error Rate for Recipe Generation:", results_CER)
print("Word Error Rate for Recipe Generation:",results_WER)
print("Exact Match for Recipe Generation:",results_em)
print("BLEU Score for Recipe Generation:",results_bleu)
print("METEOR for Recipe Generation:",results_met)

Character Error Rate for Recipe Generation: 1.846136311830408
Word Error Rate for Recipe Generation: 2.181938911022576
Exact Match for Recipe Generation: {'exact_match': 0.0}
BLEU Score for Recipe Generation: {'bleu': 0.09745879485433596, 'precisions': [0.23334679047234558, 0.10866910866910867, 0.06934755847353304, 0.05130326851468763], 'brevity_penalty': 1.0, 'length_ratio': 2.4355948869223205, 'translation_length': 2477, 'reference_length': 1017}
METEOR for Recipe Generation: {'meteor': 0.3252215076904011}


In [ ]:
from rouge_score import rouge_scorer
# Initialize the Rouge scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=False)

# Preprocess the text
def preprocess_text(text):
    tokens = text.split()
    return ' '.join(tokens)

# Calculate scores for each pair of predictions and references
rouge1_f1_scores = []
rouge1_precision_scores = []
rouge1_recall_scores = []
rouge2_f1_scores = []
rouge2_precision_scores = []
rouge2_recall_scores = []
rougeL_f1_scores = []
rougeL_precision_scores = []
rougeL_recall_scores = []

for ref, pred in zip(list_of_20_examples, list_of_generated_recipes):
    candidate = preprocess_text(pred)
    reference = preprocess_text(' '.join(ref))
    scores = scorer.score(reference, candidate)

    rouge1_f1_scores.append(scores['rouge1'].fmeasure)
    rouge1_precision_scores.append(scores['rouge1'].precision)
    rouge1_recall_scores.append(scores['rouge1'].recall)
    rouge2_f1_scores.append(scores['rouge2'].fmeasure)
    rouge2_precision_scores.append(scores['rouge2'].precision)
    rouge2_recall_scores.append(scores['rouge2'].recall)
    rougeL_f1_scores.append(scores['rougeL'].fmeasure)
    rougeL_precision_scores.append(scores['rougeL'].precision)
    rougeL_recall_scores.append(scores['rougeL'].recall)

# Calculate the average scores
avg_rouge1_f1 = sum(rouge1_f1_scores) / len(rouge1_f1_scores)
avg_rouge1_precision = sum(rouge1_precision_scores) / len(rouge1_precision_scores)
avg_rouge1_recall = sum(rouge1_recall_scores) / len(rouge1_recall_scores)
avg_rouge2_f1 = sum(rouge2_f1_scores) / len(rouge2_f1_scores)
avg_rouge2_precision = sum(rouge2_precision_scores) / len(rouge2_precision_scores)
avg_rouge2_recall = sum(rouge2_recall_scores) / len(rouge2_recall_scores)
avg_rougeL_f1 = sum(rougeL_f1_scores) / len(rougeL_f1_scores)
avg_rougeL_precision = sum(rougeL_precision_scores) / len(rougeL_precision_scores)
avg_rougeL_recall = sum(rougeL_recall_scores) / len(rougeL_recall_scores)

# Print the average scores
print("Average Rouge-1 F1 Score:", avg_rouge1_f1)
print("Average Rouge-1 Precision:", avg_rouge1_precision)
print("Average Rouge-1 Recall:", avg_rouge1_recall)

print("Average Rouge-2 F1 Score:", avg_rouge2_f1)
print("Average Rouge-2 Precision:", avg_rouge2_precision)
print("Average Rouge-2 Recall:", avg_rouge2_recall)

print("Average Rouge-L F1 Score:", avg_rougeL_f1)
print("Average Rouge-L Precision:", avg_rougeL_precision)
print("Average Rouge-L Recall:", avg_rougeL_recall)

Average Rouge-1 F1 Score: 0.029069723928599724
Average Rouge-1 Precision: 0.040044938060854365
Average Rouge-1 Recall: 0.02966973301740704
Average Rouge-2 F1 Score: 0.0037649484386785754
Average Rouge-2 Precision: 0.0037757545227833852
Average Rouge-2 Recall: 0.0050208141607210955
Average Rouge-L F1 Score: 0.02626064157835723
Average Rouge-L Precision: 0.036315605072322364
Average Rouge-L Recall: 0.02699192091551721
